In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import inf

In [191]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [192]:
colnames=['timestamp', 'game_name', 'num_channels', 'num_viewers']

In [193]:
df=pd.read_csv('Natasha.csv',header =None, names = colnames)

In [194]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [195]:
df=df.sort_values(by=['game_name','timestamp'])

In [196]:
df.reset_index(inplace = True)

In [197]:
df.drop(columns='index',inplace = True)

In [198]:
# df['diff_channels'] = df['num_channels']
# df['diff_viewers'] = df['num_viewers']
# for row_index in range(1,df.shape[0]):
#     # 0 - time_logged
#     # 1 - game_name
#     # 2 - num_channels
#     # 3 - num_viewers
#     # 4 - diff_channels
#     # 5 - diff_viewers
    
    
#     # Check if game_name of this row and previous row are the same
#     if df.iloc[row_index,1] == df.iloc[row_index-1,1]:
#         df.iloc[row_index,4] = int(df.iloc[row_index,2] - df.iloc[row_index-1,2])
#         df.iloc[row_index,5] = int(df.iloc[row_index,3] - df.iloc[row_index-1,3])
#     else:
#     # If the game names are different, the difference between metrics is invalid, fill with NaN
#         df.iloc[row_index,4] = np.NaN
#         df.iloc[row_index,5] = np.NaN

In [199]:
df['diff_channels'] = df['num_channels']
df['diff_viewers'] = df['num_viewers']

In [200]:
df_values =  df.values

In [201]:
df_values[4,1]

' F1 2019'

In [202]:
for row_index in range(1,df.shape[0]):
    # 0 - time_logged
    # 1 - game_name
    # 2 - num_channels
    # 3 - num_viewers
    # 4 - diff_channels
    # 5 - diff_viewers
    
    
    # Check if game_name of this row and previous row are the same
    if df_values[row_index,1] == df_values[row_index-1,1]:
        df_values[row_index,4] = int(df_values[row_index,2] - df_values[row_index-1,2])
        df_values[row_index,5] = int(df_values[row_index,3] - df_values[row_index-1,3])
    else:
    # If the game names are different, the difference between metrics is invalid, fill with NaN
        df_values[row_index,4] = np.NaN
        df_values[row_index,5] = np.NaN

In [205]:
diff_df=pd.DataFrame(df_values).rename(columns={0: 'timestamp',
                                                1: 'game_name',
                                                2: 'num_channels',
                                                3: 'num_viewers',
                                                4: 'diff_channels',
                                                5: 'diff_viewers'})

In [206]:
# diff_df.to_pickle('./sully_df_differences')

In [209]:
df=pd.read_pickle('./sully_df_differences')

In [210]:
df.shape

(2092036, 6)

In [204]:
index_n=50000
test_df.iloc[index_n:index_n+20]

,timestamp,game_name,num_channels,num_viewers,diff_channels,diff_viewers
50000,2019-05-28 15:00:00,Alone in the Dark,1,1,0,-1
50001,2019-05-28 16:00:00,Alone in the Dark,1,1,0,0
50002,2019-05-30 11:00:00,Alone in the Dark,1,0,0,-1
50003,2019-05-30 12:00:00,Alone in the Dark,1,3,0,3
50004,2019-05-30 13:00:00,Alone in the Dark,1,3,0,0
50005,2019-04-22 18:00:00,Alone in the Dark 2,1,1,NaN,NaN
50006,2019-04-22 19:00:00,Alone in the Dark 2,1,1,0,0
50007,2019-04-22 20:00:00,Alone in the Dark 2,1,1,0,0
50008,2019-04-22 21:00:00,Alone in the Dark 2,1,1,0,0
50009,2019-04-22 22:00:00,Alone in the Dark 2,1,1,0,0


In [46]:
def train_lstm(train, feature_names, lags, look_back, predict_window):
    dataset = np.matrix(train[feature_names])
    lstm_in_X, lstm_in_Y, features = make_variables(dataset, lags, look_back, predict_window)
    train_X = lstm_in_X
    train_Y = lstm_in_Y

    model = Sequential()
    model.add(LSTM(32, input_shape=(look_back+1, features)))
    model.add(Dropout(0.2))
    model.add(Dense(look_back+1))
    model.compile(loss='squared_hinge', optimizer='adam')
    model.fit(train_X, train_Y, epochs=300, batch_size=10, verbose=1)
    
    pred_Y_train = model.predict(train_X)
    predictions = pred_Y_train[:,-1]
    actuals = train_Y[:,-1]
    
    return model, dataset, train_X, train_Y, predictions, actuals

In [216]:
LoL=df.loc[df['game_name'] == 'League of Legends']

In [219]:
LoL.drop(columns = ['timestamp','game_name', 'num_channels','num_viewers'])

,diff_channels,diff_viewers
938390,NaN,NaN
938391,-376,-16775
938392,-83,3915
938393,-14,6393
938394,-201,18045
938395,-216,-10105
938396,-259,330
938397,-227,-19222
938398,-142,3021
938399,49,-736


In [215]:
timeseries_to_supervised(df)

,timestamp,game_name,num_channels,num_viewers,diff_channels,diff_viewers,timestamp,game_name,num_channels,num_viewers,diff_channels,diff_viewers
0,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-21 15:00:00,F1 2019,1,1,1,1
1,2019-05-21 15:00:00,F1 2019,1,1,1,1,2019-05-21 21:00:00,F1 2019,1,1,0,0
2,2019-05-21 21:00:00,F1 2019,1,1,0,0,2019-05-21 22:00:00,F1 2019,1,2,0,1
3,2019-05-21 22:00:00,F1 2019,1,2,0,1,2019-05-25 01:00:00,F1 2019,1,235,0,233
4,2019-05-25 01:00:00,F1 2019,1,235,0,233,2019-05-25 02:00:00,F1 2019,1,247,0,12
5,2019-05-25 02:00:00,F1 2019,1,247,0,12,2019-05-26 12:00:00,F1 2019,1,38,0,-209
6,2019-05-26 12:00:00,F1 2019,1,38,0,-209,2019-05-26 13:00:00,F1 2019,1,164,0,126
7,2019-05-26 13:00:00,F1 2019,1,164,0,126,2019-05-26 14:00:00,F1 2019,1,108,0,-56
8,2019-05-26 14:00:00,F1 2019,1,108,0,-56,2019-05-26 15:00:00,F1 2019,1,22,0,-86
9,2019-05-26 15:00:00,F1 2019,1,22,0,-86,2019-05-26 16:00:00,F1 2019,1,4,0,-18


# Jason Brownlee

LSTM Code

https://machinelearningmastery.com/use-features-lstm-networks-time-series-forecasting/

In [214]:
def timeseries_to_supervised(data, lag=1):
#     df = DataFrame(data)
    df=data
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    return df
 
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, yhat):
    new_row = [x for x in X] + [yhat]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]
 
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]
 
# run a repeated experiment
def experiment(repeats, series, features):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, features)
    supervised_values = supervised.values[features:,:]
    # split data into train and test-sets
    train, test = supervised_values[0:-12, :], supervised_values[-12:, :]
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    # run experiment
    error_scores = list()
    for r in range(repeats):
        # fit the base model
        lstm_model = fit_lstm(train_scaled, 1, 500, 1)
        # forecast test dataset
        predictions = list()
        for i in range(len(test_scaled)):
            # predict
            X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
            yhat = forecast_lstm(lstm_model, 1, X)
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
            # store forecast
            predictions.append(yhat)
        # report performance
        rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
        print('%d) Test RMSE: %.3f' % (r+1, rmse))
        error_scores.append(rmse)
    return error_scores
 
# execute the experiment
def run():
    # load dataset
    series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
    # experiment
    repeats = 10
    results = DataFrame()
    # run experiment
    features = 1
    results['results'] = experiment(repeats, series, features)
    # summarize results
    print(results.describe())
    # save results
    results.to_csv('experiment_features_1.csv', index=False)
 
 # entry point
# run()

# Tony's Code

In [ ]:
# Main - for running the pipeline

tickers = ['BAC','C']
train_perc = 0.40
lags = 50
look_back = 5       # 0 is a look_back of 1, 1 is a look_back of 2, etc.
predict_window = 1

# Neural net parameters
lstm_neurons = 16
dense_layers = []

In [221]:
LoL.head()

,timestamp,game_name,num_channels,num_viewers,diff_channels,diff_viewers
938390,2019-04-22 00:00:00,League of Legends,3160,97898,NaN,NaN
938391,2019-04-22 01:00:00,League of Legends,2784,81123,-376,-16775
938392,2019-04-22 02:00:00,League of Legends,2701,85038,-83,3915
938393,2019-04-22 03:00:00,League of Legends,2687,91431,-14,6393
938394,2019-04-22 04:00:00,League of Legends,2486,109476,-201,18045


In [225]:
return_df=LoL

In [226]:
slice_data(.4,['diff_viewers','diff_channels'],LoL)

KeyError: "['v_diff_channels', 'v_diff_viewers'] not in index"

In [223]:
def slice_data(train_ratio, columns_to_compare, return_df_2):
    # Divide into train set by cutting up dataframe using train_len
    train_len = int(return_df.shape[0] * (train_ratio))
    # Returns a list of all columns in columns to compare
    volume_to_trade = ['v_'+ticker for ticker in columns_to_compare]
    
    for i in volume_to_trade:
        columns_to_compare.append(i)

    train = return_df_2[columns_to_compare][1:train_len].copy()
    train['diff'] = train[columns_to_compare[0]] - train[columns_to_compare[1]]
    train.reset_index(inplace=True, drop=True)

    test = return_df_2[columns_to_compare][train_len:-400].copy()
    test['diff'] = test[columns_to_compare[0]] - test[columns_to_compare[1]]
    test.reset_index(inplace=True, drop=True)

    true_test = return_df_2[columns_to_compare][-400:].copy()
    true_test['diff'] = true_test[columns_to_compare[0]] - true_test[columns_to_compare[1]]
    true_test.reset_index(inplace=True, drop=True)

    feature_names = volume_to_trade
    feature_names.append('diff')
    
    return train, test, true_test, feature_names

In [227]:
def make_variables(dataset, lags, look_back, predict_window):
    ''' dataset = np.matrix
        lags = integer, number of previous rows to include as features for predicting next
        look_back = integer, number of previous MODELS
        predict_window = integer, number of rows to predict ahead
    '''
    num_features = lags+1
    start = look_back
    stop = len(dataset) - lags - predict_window

    lstm_in_X = np.zeros(shape=(stop-start, look_back+1, num_features))
    lstm_in_Y = np.zeros(shape=(stop-start, look_back+1))

    iter_list = [num for num in range(look_back+1)][::-1]
    for i in range(start, stop):
        for index, j in enumerate(iter_list):
            X = dataset[i-j : i-j+lags+1, -1]
            lstm_in_X[i-start, index] = np.ravel(X)
            Y = dataset[i-j+lags+1, -1]
            lstm_in_Y[i-start, index] = Y
    return lstm_in_X, lstm_in_Y, num_features

In [ ]:
def train_lstm(train, feature_names, lags, look_back, predict_window, lstm_neurons, dense_layers):
    dataset = np.matrix(train[feature_names])
    lstm_in_X, lstm_in_Y, features = make_variables(dataset, lags, look_back, predict_window)
    train_X = lstm_in_X
    train_Y = lstm_in_Y

    model = Sequential()
    model.add(LSTM(lstm_neurons, input_shape=(look_back+1, features)))
    model.add(Dropout(0.2))
    for neurons in dense_layers:
        model.add(Dense(neurons))
        model.add(Dropout(0.2))
    model.add(Dense(look_back+1))
    model.compile(loss='squared_hinge', optimizer='adam')
    model.fit(train_X, train_Y, epochs=300, batch_size=10, verbose=1)
    
    pred_Y_train = model.predict(train_X)
    predictions = pred_Y_train[:,-1]
    actuals = train_Y[:,-1]
    
    print(model.summary())
    
    return model, dataset, train_X, train_Y, predictions, actuals